In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import kagglehub

# Download latest version
path = kagglehub.dataset_download("rjmanoj/credit-card-customer-churn-prediction")

print("Path to dataset files:", path)

/kaggle/input/credit-card-customer-churn-prediction/Churn_Modelling.csv
Path to dataset files: /kaggle/input/credit-card-customer-churn-prediction


In [2]:
df = pd.read_csv('/kaggle/input/credit-card-customer-churn-prediction/Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df.shape

(10000, 14)

In [4]:
df.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [5]:
df.duplicated().sum()

0

In [6]:
df['Exited'].value_counts()

Exited
0    7963
1    2037
Name: count, dtype: int64

In [7]:
df['Geography'].value_counts()

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [8]:
df.drop(columns=['RowNumber', 'CustomerId', 'Surname'], inplace=True)

In [9]:
dummies = pd.get_dummies(df[['Geography', 'Gender']], drop_first=True, dtype=int)

In [10]:
df = pd.concat([df.drop(['Geography', 'Gender'], axis=1), dummies], axis=1)

In [11]:
df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1,0


In [12]:
X = df.drop(columns=['Exited'])
y = df['Exited']

In [13]:
y = df['Exited'].values

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
X

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0,0,1
9996,516,35,10,57369.61,1,1,1,101699.77,0,0,1
9997,709,36,7,0.00,1,0,1,42085.58,0,0,0
9998,772,42,3,75075.31,2,1,0,92888.52,1,0,1


In [16]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim

In [18]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

In [19]:
class SimpleANN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleANN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

# Initialize model
input_dim = X_train.shape[1]
hidden_dim = 8
output_dim = 1
model = SimpleANN(input_dim, hidden_dim, output_dim)

In [20]:
criterion = nn.BCELoss()  # Binary Cross Entropy for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [21]:
epochs = 100
for epoch in range(epochs):
    model.train()  # training mode

    # Forward pass (compute outputs fresh each epoch)
    outputs = model(X_train_tensor)

    # Compute loss
    loss = criterion(outputs, y_train_tensor)

    # Zero gradients before backward pass
    optimizer.zero_grad()

    # Backward pass
    loss.backward()  # called only once per forward

    # Update weights
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [10/100], Loss: 0.6735
Epoch [20/100], Loss: 0.6573
Epoch [30/100], Loss: 0.6422
Epoch [40/100], Loss: 0.6281
Epoch [50/100], Loss: 0.6149
Epoch [60/100], Loss: 0.6025
Epoch [70/100], Loss: 0.5910
Epoch [80/100], Loss: 0.5802
Epoch [90/100], Loss: 0.5701
Epoch [100/100], Loss: 0.5606


In [22]:
model.eval()  # set model to evaluation mode
with torch.no_grad():
    y_pred = model(X_test_tensor)
    y_pred_cls = (y_pred > 0.5).float()

In [23]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# Make predictions on test set
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
    y_pred_cls = (y_pred > 0.5).float()

# Convert to numpy for sklearn metrics
y_test_np = y_test_tensor.numpy()
y_pred_np = y_pred_cls.numpy()

# Metrics
acc = accuracy_score(y_test_np, y_pred_np)
cm = confusion_matrix(y_test_np, y_pred_np)
report = classification_report(y_test_np, y_pred_np)

print("\nAccuracy:", acc)
print("\nConfusion Matrix:\n", cm)
print("\nClassification Report:\n", report)



Accuracy: 0.8105

Confusion Matrix:
 [[1594   13]
 [ 366   27]]

Classification Report:
               precision    recall  f1-score   support

         0.0       0.81      0.99      0.89      1607
         1.0       0.68      0.07      0.12       393

    accuracy                           0.81      2000
   macro avg       0.74      0.53      0.51      2000
weighted avg       0.79      0.81      0.74      2000

